This notebook provides an example code for the orientation correction step.

**Important Note:**  It's recommended to refer to the [Getting Oriented using the Image Plane Module](http://dicomiseasy.blogspot.com/2013/06/getting-oriented-using-image-plane.html) article along with the notebook reading.

The example code uses one image from the [TCIA ACRIN 6664 (CT COLONOGRAPHY) Dataset](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=3539213).

*Data Citation:*
Smith K, Clark K, Bennett W, Nolan T, Kirby J, Wolfsberger M, Moulton J, Vendt B, Freymann J. (2015). Data From CT_COLONOGRAPHY. The Cancer Imaging Archive. https://doi.org/10.7937/K9/TCIA.2015.NWTESAY1

*Publication Citation:*
Johnson, C. D., Chen, M.-H., Toledano, A. Y., Heiken, J. P., Dachman, A., Kuo, M. D., … Limburg, P. J. (2008, September 18). Accuracy of CT Colonography for Detection of Large Adenomas and Cancers. New England Journal of Medicine. New England Journal of Medicine (NEJM/MMS). https://doi.org/10.1056/nejmoa0800996

*TCIA Citation:*
Clark K, Vendt B, Smith K, Freymann J, Kirby J, Koppel P, Moore S, Phillips S, Maffitt D, Pringle M, Tarbox L, Prior F. The Cancer Imaging Archive (TCIA): Maintaining and Operating a Public Information Repository, Journal of Digital Imaging, Volume 26, Number 6, December, 2013, pp 1045-1057. https://doi.org/10.1007/s10278-013-9622-7

In [ ]:
import numpy as np
import pydicom

import matplotlib.pyplot as plt

You can see the Feet First Decubitus Right image below and its corresponding tags.
The orientation-description letters for the image should be located like this:  
&nbsp;&nbsp;&nbsp;&nbsp;R  
A&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;P  
&nbsp;&nbsp;&nbsp;&nbsp;L  
If you open this DICOM file in a viewer you'll see exactly the same image and orientation-description letters.

In [ ]:
filepath = './test_files/ct_example.dcm'
dataset= pydicom.dcmread(filepath)
img = dataset.pixel_array
img = pydicom.pixel_data_handlers.apply_modality_lut(img, dataset)
print('Image Orientation Patient:', dataset.get('ImageOrientationPatient'))
print('Patient Position:', dataset.get('PatientPosition'))
plt.imshow(img, cmap='gray')
plt.show()

Now let's correct the image according to the Image Orientation Patient tag.  

Firstly, we should make the Image Orientation Patient row and column vectors to be a right-handed pair. Otherwise, all the transforms would be incorrect.
To do so, we need to calculate a cross-product of the two vectors and check if the Z-value of the result is positive or negative.
A positive value means the pair is right-handed, negative &ndash; left-handed.
If the pair is left-handed, one of the vectors should be inverted (and the image should be mirrored accordingly).  

Then, we need to align the RCS (Reference Coordinate System) axis with the image axis to be parallel (but not necessary in the same direction, yet).
In this case, we should rotate the image once by 90&deg;

Lastly, we should align the directions of the axis. Mirror the image across the right direction if needed.  


The function below does all required checks and transformations. Also, it makes prints and visualizations of every step for clarity.

In [ ]:
def correct_iop(img: np.ndarray, ds: pydicom.dataset.FileDataset, filepath: str) -> np.ndarray:
    """
    Transforms slice according to Image Orientation Patient (0020,0032) tag:
    https://dicom.innolitics.com/ciods/ct-image/image-plane/00200037

    :param img: An input image to transform;
    :param ds: A pydicom dataset containing metadata;
    :param filepath: A path to the file, required by _logger to return meaningful message;
    :return: Corrected image as a numpy array.
    """
    iop = [float(x) for x in ds.get('ImageOrientationPatient')]
    if len(set([float(_) for _ in iop]).difference(set([0., 1., -1.]))) > 0:
        _logger.warning('File %s header contains transformations that can not be applied,\
                         skipping orientation correction step.', filepath)
        return img
    
    iop_row = np.asarray(iop[:3])
    iop_col = np.asarray(iop[3:])
    print('Input:', iop_row, iop_col)
    plt.imshow(img, cmap='gray')
    plt.show()
    
    def is_right_handed(a, b):
        # cross product
        # x = a[1] * b[2] - a[2] * b[1]
        # y = a[2] * b[0] - a[0] * b[2]
        z = a[0] * b[1] - a[1] * b[0]
        if z < 0:
            return False
        return True
    
    # Make right-handed pair of vectors
    if not is_right_handed(iop_row, iop_col):  # if left-handed, flip horizontally 
        iop_row *= -1
        img = np.flip(img, 1)
    print('Make right-handed:', iop_row, iop_col)
    plt.imshow(img, cmap='gray')
    plt.show()
    
    # Rotate to align axis
    rot90_mtx = [[0, -1, 0], [1, 0, 0], [0, 0, 1]]
    if iop_row[0] == 0:
        img = np.rot90(img)
        iop_row = np.matmul(rot90_mtx, iop_row)
        iop_col = np.matmul(rot90_mtx, iop_col)
    print('Rotate:', iop_row, iop_col)
    plt.imshow(img, cmap='gray')
    plt.show()    
    
    # Mirror to align directions
    if iop_row[0] == -1:
        iop_row *= -1
        img = np.flip(img, 1)
    if iop_col[1] == -1:
        iop_col *= -1
        img = np.flip(img, 0)
    print('Mirror:', iop_row, iop_col)
    plt.imshow(img, cmap='gray')
    plt.show()

    return img

In [ ]:
img = correct_iop(img, dataset, filepath)

Now, let's rotate the image to be oriented according to gravity (when scanner table is at the bottom of the image).

In [ ]:
def correct_pp(img: np.ndarray, ds: pydicom.dataset.FileDataset, filepath: str) -> np.ndarray:
    """
    Transforms slice according to Patient Position (0018,5100) tag:
    https://dicom.innolitics.com/ciods/ct-image/general-series/00185100

    :param img: An input image to transform;
    :param ds: A pydicom dataset containing metadata;
    :param filepath: A path to the file, required by _logger to return meaningful message;
    :return: Corrected image as a numpy array.
    """
    pp = ds.get('PatientPosition')
    if pp not in ['HFS', 'HFP', 'HFDR', 'DFDL', 'FFS', 'FFP', 'FFDR', 'FFDL']:
        _logger.warning('Unknown PatientPosition found: %s in file %s. Skipping rotation according to gravity', str(pp), filepath)
        return img
    if pp.endswith('S'):  # supine (HFS, FFS)
        return img
    if pp.endswith('P'):  # prone (HFP, FFP)
        return np.rot90(img, k=2)
    if pp.endswith('DR'):  # decubitus right (HFDR, FFDR)
        return np.rot90(img, k=1)
    if pp.endswith('DL'):  # decubitus left (HFDL, FFDL)
        return np.rot90(img, k=3)

In [ ]:
img = correct_pp(img, dataset, filepath)
plt.imshow(img, cmap='gray')
plt.show()